In [107]:
import os
import pandas as pd
import requests
from sqlalchemy import create_engine, insert, text

In [91]:
etherscan_token = os.environ.get('ETHERSCAN_TOKEN')
pepe_address = '0x6982508145454ce325ddbe47a25d4ec3d2311933'

## TokenTX

### Starting with an example

In [58]:
url = 'https://api.etherscan.io/api?module=account&action=tokentx&contractaddress=' + pepe_address + '&startblock='+ block + '&endblock=99999999&page=1&offset=1000&sort=asc&apikey=' + etherscan_token

pepe_transactions = pd.read_json(url)

pepe_transactions

,status,message,result
0,1,OK,"{'blockNumber': '17046105', 'timeStamp': '1681..."
1,1,OK,"{'blockNumber': '17046538', 'timeStamp': '1681..."
2,1,OK,"{'blockNumber': '17046833', 'timeStamp': '1681..."
3,1,OK,"{'blockNumber': '17046900', 'timeStamp': '1681..."
4,1,OK,"{'blockNumber': '17047545', 'timeStamp': '1681..."
...,...,...,...
995,1,OK,"{'blockNumber': '17050649', 'timeStamp': '1681..."
996,1,OK,"{'blockNumber': '17050656', 'timeStamp': '1681..."
997,1,OK,"{'blockNumber': '17050672', 'timeStamp': '1681..."
998,1,OK,"{'blockNumber': '17050681', 'timeStamp': '1681..."


### Iterating through the blocks, starting from the beginning and saving into DB

In [114]:
i = 0
starting_block = '17046105'
block = starting_block

pepe_transactions = []

while i<100:
    url = 'https://api.etherscan.io/api?module=account&action=tokentx&contractaddress=' + pepe_address + '&startblock='+ block + '&endblock=99999999&page=1&offset=1000&sort=asc&apikey=' + etherscan_token
    pepe_excerpt = requests.get(url).json()['result']
    pepe_transactions.extend(pepe_excerpt)
    block = pepe_excerpt[-1]['blockNumber']
    i += 1

pepe_transactions = pd.DataFrame(pepe_transactions)
pepe_transactions['timeStamp'] = pd.to_datetime(pepe_transactions['timeStamp'], unit='s')

pepe_transactions

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,tokenSymbol,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations
0,17046105,2023-04-14 14:51:35,0x2afae7763487e60b893cb57803694810e6d3d136186a...,0,0xa4cf20c9c9e367e225464eba8dc7a6bb6af4de523144...,0x0000000000000000000000000000000000000000,0x6982508145454ce325ddbe47a25d4ec3d2311933,0xfbfeaf0da0f2fde5c66df570133ae35f3eb58c9a,420690000000000000000000000000000,Pepe,PEPE,18,80,1173624,31537151435,1173624,6190821,deprecated,3525171
1,17046538,2023-04-14 16:20:47,0xb38bba8bf8f61029a31e027c37fbc1065db784a39501...,3,0xc58f8a01c65f86b938716fc04652ea0aaf534828e4ba...,0xfbfeaf0da0f2fde5c66df570133ae35f3eb58c9a,0x6982508145454ce325ddbe47a25d4ec3d2311933,0x92fb5b4f8030103e0b11275c30965d1897ff23e5,29027610000000000000000000000000,Pepe,PEPE,18,149,90858,30726614366,60572,12843112,deprecated,3524738
2,17046833,2023-04-14 17:21:11,0x273894b35d8c30d32e1ffa22ee6aa320cc9f55f2adbb...,4,0xdeb266f17a263a050035bd68619c928955a01deaeaa4...,0xfbfeaf0da0f2fde5c66df570133ae35f3eb58c9a,0x6982508145454ce325ddbe47a25d4ec3d2311933,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,391662390000000000000000000000000,Pepe,PEPE,18,182,3313678,28560581340,2714900,14593006,deprecated,3524443
3,17046900,2023-04-14 17:34:47,0xc3760b499850b19d8f266dc10349d70f69c916982336...,5,0xef6e1febb412218cea3663d889df5b4dc0c3c10515b7...,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,0x6982508145454ce325ddbe47a25d4ec3d2311933,0xfbfeaf0da0f2fde5c66df570133ae35f3eb58c9a,195146420924169301603150829311,Pepe,PEPE,18,214,266254,25335921453,175690,16011887,deprecated,3524376
4,17047545,2023-04-14 19:46:59,0x08153c0546e4f73b178edae90d1e30fb519c7c49ff0d...,12056,0xb4b290a4a0225462195468584a232784215fd1326bb2...,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,0x6982508145454ce325ddbe47a25d4ec3d2311933,0xaf2358e98683265cbd3a48509123d390ddf54534,2519324010963419102704251293024,Pepe,PEPE,18,1,237736,73437547643,132789,341269,deprecated,3523731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,17080636,2023-04-19 12:26:23,0x5cabf7c4f75022606cc3bb2282ba510bc0a8c26fa070...,8,0x261d8afba1e3bb4bfd4060834a01df2f030ac8b25fac...,0xb3e31f884822c6bf3a3e648fdb4f3d8f88bf364b,0x6982508145454ce325ddbe47a25d4ec3d2311933,0x38358b228042be593197ba558ea9d04bee8252bc,8233980049118891681799484263,Pepe,PEPE,18,119,90765,71736389682,55710,11130592,deprecated,3490649
99996,17080637,2023-04-19 12:26:35,0xa955e1c1ad9a2ed4d6a2130d098e5410b527a94a340c...,954,0x749296a3170314121e3ce6ed894861664098397ad3db...,0x1d97cb879315be7d6a7dab5f687a53149f12eecd,0x6982508145454ce325ddbe47a25d4ec3d2311933,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,13276656898000000000000000000,Pepe,PEPE,18,89,154992,71623945133,126736,7910623,deprecated,3490648
99997,17080637,2023-04-19 12:26:35,0xfd289656f3b724118cb787da715451ef02ec77c92779...,4055,0x749296a3170314121e3ce6ed894861664098397ad3db...,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,0x6982508145454ce325ddbe47a25d4ec3d2311933,0x555b6ee8fab3dfdbcca9121721c435fd4c7a1fd1,389996436672129881709244017,Pepe,PEPE,18,97,319305,71405572440,232222,8945327,deprecated,3490648
99998,17080637,2023-04-19 12:26:35,0xfd289656f3b724118cb787da715451ef02ec77c92779...,4055,0x749296a3170314121e3ce6ed894861664098397ad3db...,0x555b6ee8fab3dfdbcca9121721c435fd4c7a1fd1,0x6982508145454ce325ddbe47a25d4ec3d2311933,0x6131b5fae19ea4f9d964eac0408e4408b66337b5,389996436672129881709244017,Pepe,PEPE,18,97,319305,71405572440,232222,8945327,deprecated,3490648


In [115]:
engine=create_engine('sqlite:///../data/etherscan.db',echo=True)
sqlite_connection=engine.connect()
sqlite_table='etherscan_tokentx'
pepe_transactions.to_sql(sqlite_table,engine, if_exists='fail', index=False)
sqlite_connection.close()

2024-08-20 18:20:26,600 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 18:20:26,601 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("etherscan_tokentx")
2024-08-20 18:20:26,602 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-20 18:20:26,606 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("etherscan_tokentx")
2024-08-20 18:20:26,608 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-20 18:20:26,609 INFO sqlalchemy.engine.Engine ROLLBACK
2024-08-20 18:20:26,612 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 18:20:26,616 INFO sqlalchemy.engine.Engine 
CREATE TABLE etherscan_tokentx (
	"blockNumber" TEXT, 
	"timeStamp" DATETIME, 
	hash TEXT, 
	nonce TEXT, 
	"blockHash" TEXT, 
	"from" TEXT, 
	"contractAddress" TEXT, 
	"to" TEXT, 
	value TEXT, 
	"tokenName" TEXT, 
	"tokenSymbol" TEXT, 
	"tokenDecimal" TEXT, 
	"transactionIndex" TEXT, 
	gas TEXT, 
	"gasPrice" TEXT, 
	"gasUsed" TEXT, 
	"cumulativeGasUsed" TEXT, 
	input TEXT, 
	confirmations TEXT
)


2024-08-2

### Starting from wherever we left off and appending to the DB

In [110]:
# Retrieve the block number of the last row on the table in etherscan.db

engine=create_engine('sqlite:///../data/etherscan.db',echo=True)
sqlite_connection=engine.connect()
sqlite_table='etherscan_tokentx'
sql_query = f'''SELECT * FROM {sqlite_table} ORDER BY blockNumber DESC LIMIT 1'''

last_block = pd.read_sql(text(sql_query), sqlite_connection)['blockNumber'].values[0]

sqlite_connection.close()

2024-08-20 18:12:37,127 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 18:12:37,131 INFO sqlalchemy.engine.Engine SELECT * FROM etherscan_tokentx ORDER BY blockNumber DESC LIMIT 1
2024-08-20 18:12:37,132 INFO sqlalchemy.engine.Engine [generated in 0.00529s] ()
2024-08-20 18:12:37,144 INFO sqlalchemy.engine.Engine ROLLBACK


'17056276'

In [111]:
i = 0
block = last_block

pepe_transactions_2 = []

while i<3:
    url = 'https://api.etherscan.io/api?module=account&action=tokentx&contractaddress=' + pepe_address + '&startblock='+ block + '&endblock=99999999&page=1&offset=1000&sort=asc&apikey=' + etherscan_token
    pepe_excerpt = requests.get(url).json()['result']
    pepe_transactions_2.extend(pepe_excerpt)
    block = pepe_excerpt[-1]['blockNumber']
    i += 1

pepe_transactions_2 = pd.DataFrame(pepe_transactions_2)
pepe_transactions_2['timeStamp'] = pd.to_datetime(pepe_transactions_2['timeStamp'], unit='s')

pepe_transactions_2

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,tokenSymbol,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations
0,17056276,2023-04-16 01:39:11,0xdee4d9db902cad4e98f9ef0f0143150a10c7dd96420d...,161829,0x9168356abd4f315728d1d2fa2bc7b65262de7b1ac6ac...,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,0x6982508145454ce325ddbe47a25d4ec3d2311933,0x6b75d8af000000e20b7a7ddf000ba900b4009a80,188118722555619014273132396544,Pepe,PEPE,18,0,240592,24341935923,168415,168415,deprecated,3514979
1,17056276,2023-04-16 01:39:11,0xf12c8b51deb51362d40d72e9ea74eba75fcf52fea43d...,747,0x9168356abd4f315728d1d2fa2bc7b65262de7b1ac6ac...,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,0x6982508145454ce325ddbe47a25d4ec3d2311933,0x479b0e651693aed68fe42ae60bd5183da253bcda,422983460870041807551523142993,Pepe,PEPE,18,1,256042,24441935923,192361,360776,deprecated,3514979
2,17056276,2023-04-16 01:39:11,0x1be3251a512197a59e9ad0d5573f17d0134061d3942e...,161830,0x9168356abd4f315728d1d2fa2bc7b65262de7b1ac6ac...,0x6b75d8af000000e20b7a7ddf000ba900b4009a80,0x6982508145454ce325ddbe47a25d4ec3d2311933,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,188118722555619014273132396544,Pepe,PEPE,18,3,188325,147688507930,131828,628006,deprecated,3514979
3,17056278,2023-04-16 01:39:35,0x6b832fc9430cf032a49846659936a26b05680e823d62...,961,0x02a9767f8e6cbff124512f03ccc6c856e549ed896c2b...,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,0x6982508145454ce325ddbe47a25d4ec3d2311933,0x0535bec48152c637d0577c5ae4fa72b516a242fe,48641561245799448909725342555,Pepe,PEPE,18,89,187592,22864259641,121762,5717681,deprecated,3514977
4,17056279,2023-04-16 01:39:47,0xb9b8977e7b408b294c1a8e022239bc25d48c2ff125e4...,265,0x65a328171ff0517412dc58f287a66a2dd507a58d97c6...,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,0x6982508145454ce325ddbe47a25d4ec3d2311933,0x555b6ee8fab3dfdbcca9121721c435fd4c7a1fd1,28617803555603150296614040506,Pepe,PEPE,18,30,319476,24441935920,240680,1519402,deprecated,3514976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,17061136,2023-04-16 18:12:35,0xfea4b2d46c0ea42a63abcaccec05e2497b0dd09eb7f6...,201,0x1135d753d7486195047c387b005994a73cc4c7cc2d78...,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,0x6982508145454ce325ddbe47a25d4ec3d2311933,0x139accf1f7e918c1061e777fe55c63d47c4f7f0d,4643676788089364981943446746,Pepe,PEPE,18,103,266706,24851734830,196447,9524885,deprecated,3510119
2996,17061137,2023-04-16 18:12:47,0x2b7cbcebd2c1d01c4c39490ff32f8804bd759a8646e9...,1293,0xd0423493fb1b780e1fe9bb32548a2be8a802cef029e3...,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,0x6982508145454ce325ddbe47a25d4ec3d2311933,0x7ba2e14c0329aa727a41c801a1fc35d46213f983,51617400091843374366535210095,Pepe,PEPE,18,115,212292,24189211745,148108,16191711,deprecated,3510118
2997,17061137,2023-04-16 18:12:47,0x82ca80a65b1fc05238fed5319b59d8659c9348272356...,35,0xd0423493fb1b780e1fe9bb32548a2be8a802cef029e3...,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,0x6982508145454ce325ddbe47a25d4ec3d2311933,0x696665a159323b2ef46b38d6c2493e83b6597e2a,9716844025649444810230452435,Pepe,PEPE,18,124,204228,24183211745,126066,16939272,deprecated,3510118
2998,17061139,2023-04-16 18:13:11,0xea498d01a012d067143c19a6c2d15b12df6c47c432f6...,211,0xa3a5f62c9e2cf24b5adc889600b81ef63b030d704885...,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,0x6982508145454ce325ddbe47a25d4ec3d2311933,0xe1b1567a02daa679b0706f33521dcc119be269d0,19394141020811088025645118519,Pepe,PEPE,18,129,204256,24160353675,134418,9581018,deprecated,3510116


In [113]:
engine=create_engine('sqlite:///../data/etherscan.db',echo=True)
sqlite_connection=engine.connect()
sqlite_table='etherscan_tokentx'
pepe_transactions_2.to_sql(sqlite_table,engine, if_exists='append', index=False)
sqlite_connection.close()

2024-08-20 18:14:41,752 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 18:14:41,753 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("etherscan_tokentx")
2024-08-20 18:14:41,754 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-20 18:14:41,757 INFO sqlalchemy.engine.Engine ROLLBACK


2024-08-20 18:14:41,775 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 18:14:41,992 INFO sqlalchemy.engine.Engine INSERT INTO etherscan_tokentx ("blockNumber", "timeStamp", hash, nonce, "blockHash", "from", "contractAddress", "to", value, "tokenName", "tokenSymbol", "tokenDecimal", "transactionIndex", gas, "gasPrice", "gasUsed", "cumulativeGasUsed", input, confirmations) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2024-08-20 18:14:41,993 INFO sqlalchemy.engine.Engine [generated in 0.20658s] [('17056276', '2023-04-16 01:39:11.000000', '0xdee4d9db902cad4e98f9ef0f0143150a10c7dd96420d65869f4a874341f560d3', '161829', '0x9168356abd4f315728d1d2fa2bc7b65262de7b1ac6ac3aa69a4950aac78e5765', '0xa43fe16908251ee70ef74718545e4fe6c5ccec9f', '0x6982508145454ce325ddbe47a25d4ec3d2311933', '0x6b75d8af000000e20b7a7ddf000ba900b4009a80', '188118722555619014273132396544', 'Pepe', 'PEPE', '18', '0', '240592', '24341935923', '168415', '168415', 'deprecated', '3514979'), ('170562

### PEPE Transaction List

In [25]:
url ='https://api.etherscan.io/api?module=account&action=txlist&address='+ pepe_address + '&startblock=0&endblock=99999999&page=2offset=10&sort=des&apikey=' + etherscan_token

pepe_txn = pd.read_json(url)

pepe_txn

,status,message,result
0,1,OK,"{'blockNumber': '17046105', 'timeStamp': '1681..."
1,1,OK,"{'blockNumber': '17046152', 'timeStamp': '1681..."
2,1,OK,"{'blockNumber': '17046538', 'timeStamp': '1681..."
3,1,OK,"{'blockNumber': '17047544', 'timeStamp': '1681..."
4,1,OK,"{'blockNumber': '17047547', 'timeStamp': '1681..."
...,...,...,...
9995,1,OK,"{'blockNumber': '17075182', 'timeStamp': '1681..."
9996,1,OK,"{'blockNumber': '17075183', 'timeStamp': '1681..."
9997,1,OK,"{'blockNumber': '17075183', 'timeStamp': '1681..."
9998,1,OK,"{'blockNumber': '17075183', 'timeStamp': '1681..."


In [16]:
pepe_txn['result'][0]

{'blockNumber': '17046105',
 'timeStamp': '1681483895',
 'hash': '0x2afae7763487e60b893cb57803694810e6d3d136186a6de6719921afd7ca304a',
 'nonce': '0',
 'blockHash': '0xa4cf20c9c9e367e225464eba8dc7a6bb6af4de523144fd400345a0467c7d7041',
 'transactionIndex': '80',
 'from': '0xfbfeaf0da0f2fde5c66df570133ae35f3eb58c9a',
 'to': '',
 'value': '0',
 'gas': '1173624',
 'gasPrice': '31537151435',
 'isError': '0',
 'txreceipt_status': '1',
 'input': '0x60806040523480156200001157600080fd5b506040516200174038038062001740833981016040819052620000349162000454565b604051806040016040528060048152602001635065706560e01b815250604051806040016040528060048152602001635045504560e01b8152506200008862000082620000cf60201b60201c565b620000d3565b81516200009d906004906020850190620003ae565b508051620000b3906005906020840190620003ae565b505050620000c833826200012360201b60201c565b506200058b565b3390565b600080546001600160a01b038381166001600160a01b0319831681178455604051919092169283917f8be0079c531659141344cd1fd0a4f28419497f9722a3daafe

In [46]:
# Get the time of the first transaction and convert to timestamp

import datetime

timestamp = pepe_txn['result'][1]['timeStamp']

timestamp = int(timestamp)

timestamp = datetime.datetime.fromtimestamp(timestamp)

timestamp

datetime.datetime(2023, 4, 14, 16, 1, 11)

In [29]:
# Get the blockNumber of the last transaction

blockNumber = pepe_txn['result'][9999]['blockNumber']
blockNumber

'17075184'

In [41]:
blockNumber2 = '20569308'

In [43]:
url ='https://api.etherscan.io/api?module=account&action=txlist&address='+ pepe_address + '&startblock='+ blockNumber2 + '&endblock=99999999&page=1offset=10&sort=des&apikey=' + etherscan_token

pepe_txn_suite = pd.read_json(url)

pepe_txn_suite

,status,message,result
0,1,OK,"{'transactionIndex': '74', 'from': '0xb2af0151..."
1,1,OK,"{'transactionIndex': '81', 'from': '0xa6c5804d..."
2,1,OK,"{'transactionIndex': '106', 'from': '0xd21258e..."
3,1,OK,"{'transactionIndex': '24', 'from': '0x64864c98..."
4,1,OK,"{'transactionIndex': '18', 'from': '0xefa34056..."
...,...,...,...
540,1,OK,"{'transactionIndex': '138', 'from': '0xdfd5293..."
541,1,OK,"{'transactionIndex': '38', 'from': '0x450969ca..."
542,1,OK,"{'transactionIndex': '30', 'from': '0x450969ca..."
543,1,OK,"{'transactionIndex': '31', 'from': '0x450969ca..."


In [49]:
# Get the time of the first transaction and convert to timestamp

import datetime

timestamp = pepe_txn_suite['result'][1]['timeStamp']

timestamp = int(timestamp)

timestamp = datetime.datetime.fromtimestamp(timestamp)

timestamp

datetime.datetime(2024, 8, 20, 11, 42, 35)

In [53]:
pepe_txn_suite['result'][543]

{'transactionIndex': '31',
 'from': '0x450969ca0ffb147667d9d2ffae3245cef5450192',
 'blockNumber': '20570249',
 'to': '0x6982508145454ce325ddbe47a25d4ec3d2311933',
 'blockHash': '0xcd3bc747b139fc6a62860bfc5729d23acc25e8a136be5581e7ccb17ab4c2f7b1',
 'timeStamp': '1724161919',
 'hash': '0x39a497cc1ee20067fe31502ef78ed0e95270c9b2d48117a9b3edfd1732845c32',
 'nonce': '6',
 'value': '0',
 'gas': '43374',
 'gasPrice': '3300000000',
 'input': '0xa9059cbb000000000000000000000000abc6895a47137fcfb6c4dfe974a31e33866dd38b00000000000000000000000000000000000000000000c5ed8f3ce470a7cf0e00',
 'methodId': '0xa9059cbb',
 'functionName': 'transfer(address _to, uint256 _value)',
 'contractAddress': '',
 'cumulativeGasUsed': '1235414',
 'txreceipt_status': '0',
 'gasUsed': '33324',
 'confirmations': '4',
 'isError': '1'}